# Кодирование методом Фибоначчи


Используем ряд чисел Фибонначи как основания псевдо-ЧС.
Естественным терминатором закодированной последовательности является **"11"**

Рекомендую использовать классы для работы с битовыми полями: BitstreamWriter и BitstreamRader.

In [1]:
class BitstreamWriter:
    def __init__(self):
        self.nbits  = 0
        self.curbyte = 0
        self.vbytes = []

    """ add single bit """
    def add(self, x):
        self.curbyte |= x << (8-1 - (self.nbits % 8))
        self.nbits += 1

        if self.nbits % 8 == 0:
            self.vbytes.append(chr(self.curbyte))
            self.curbyte = 0

    """ get byte-aligned bits """
    def getbytes(self):
        if self.nbits & 7 == 0:
            return "".join(self.vbytes)

        return "".join(self.vbytes) + chr(self.curbyte)


class BitstreamReader:
    def __init__(self, blob):
        self.blob = blob
        self.pos  = 0

    """ extract next bit """
    def get(self):
        ibyte = self.pos / 8
        ibit  = self.pos & 7

        self.pos += 1
        return (ord(self.blob[ibyte]) & (1 << (7 - ibit))) >> (7 - ibit)

    def finished(self):
        return self.pos >= len(self.blob) * 8


# Код компрессии Фибоначчи

Вы должны реализовать тела 2х функций - сжатия списка документов (dl) и его распаковки обратно - в список.  
Корректность реализации проверяется набором unit-test-ов. Успешное прохождение Unit-test-ов гарантирует правильность реализации.

In [2]:
import unittest
import random

# Use this numbers as base to encode
fib_nums = None

def init_fibnumbers():
    global fib_nums
    fib_nums = [1, 2]

    while True:
        fib_nums.append(fib_nums[-2] + fib_nums[-1])
        if fib_nums[-1] > 2**28:
            break
            
init_fibnumbers()


"""
Input dl contains monotonically groving integers
"""
def compress_fblist(dl):
    bs = BitstreamWriter()
    
    """ Write your code here """
    
    return bs.getbytes()


def decompress_fblist(s):
    bs = BitstreamReader(s)
    dl = []
    
    """ Write your code here """

    return dl


class TestFibbonacciCompression(unittest.TestCase):
    def test_simple_compression(self):
        cb = compress_fblist([4, 48, 115, 190])
        nums = decompress_fblist(cb)
        self.assertListEqual(nums, [4, 48, 115, 190])

    def test_one_element(self):
        cb = compress_fblist([40])
        nums = decompress_fblist(cb)
        self.assertListEqual(nums, [40])
        
    def test_random_elements(self):
        n = random.randint(1000, 2000)
        arr = []
        for i in xrange(n):
            delta = random.randint(1, 100)
            arr.append(delta if i == 0 else arr[i-1]+delta)
        cb = compress_fblist(arr)
        decoded = decompress_fblist(cb)
        self.assertListEqual(decoded, arr)

suite = unittest.TestLoader().loadTestsFromTestCase(TestFibbonacciCompression)
unittest.TextTestRunner().run(suite)

FFF
FAIL: test_one_element (__main__.TestFibbonacciCompression)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-2-44b6ddda348c>", line 48, in test_one_element
    self.assertListEqual(nums, [40])
AssertionError: Lists differ: [] != [40]

Second list contains 1 additional elements.
First extra element 0:
40

- []
+ [40]

FAIL: test_random_elements (__main__.TestFibbonacciCompression)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-2-44b6ddda348c>", line 58, in test_random_elements
    self.assertListEqual(decoded, arr)
AssertionError: Lists differ: [] != [100, 128, 173, 187, 202, 285,...

Second list contains 1501 additional elements.
First extra element 0:
100

Diff is 14797 characters long. Set self.maxDiff to None to see it.

FAIL: test_simple_compression (__main__.TestFibbonacciCompression)
---------------------------------

<unittest.runner.TextTestResult run=3 errors=0 failures=3>

# Сравнение с generic сжатием

Сжатие, которое вы реализовали существенно более эффективно для монотоного ряда чисел.  
Рассмотрим пример: давайте сожмем файл с большим кол-вом чисел и сравним *Fibonnaci vs Gzip*.

In [3]:
import gzip
import random
import cStringIO

def gen_numbers(n, min_step=1, max_step=5):
    nums = [1]
    for i in xrange(n):
        nums.append(nums[-1] + random.randint(min_step, max_step))   
    return nums
        
def compress_gzip(nums):
    sstream = cStringIO.StringIO()
    zs = gzip.GzipFile(fileobj=sstream, mode='wb')
    for n in nums:
        zs.write("%d\n" % n)
    zs.flush()
    return sstream.getvalue()

# поэкспериментируйте со значениями max_step
nums = gen_numbers(n=10**5, max_step=5)
gzip_len = len(compress_gzip(nums))
fblist_len = len(compress_fblist(nums))

if not fblist_len:
    raise RuntimeError("Fibonacci encoding made empty string")

print "Compressed with gzip: %d Kb" % (gzip_len / 1024)
print "Compressed with fibonacci: %d Kb" % (fblist_len / 1024)
print "Gzip/Fibonacci ratio: %.1f" % (float(gzip_len) / fblist_len)

RuntimeError: Fibonacci encoding made empty string